# Import and clean restaurant data

In [247]:
getwd()
setwd("raw-output")

[1] "C:/Users/wue04/Box Sync/tacobell/raw-output"

### suppress warnings messages

In [139]:
current_warning <- getOption("warn")
options(warn = -1)

### reset warning to normal

In [ ]:
options(warn = current_warning)

### read sample data

In [37]:
sample <- read.csv("restaurants.csv", sep = ";", header = FALSE, nrow=10, quote = "")
head(sample)
sapply(sample, class)

### read in data

In [249]:
# clean up restaurants
restaurants <- read.csv("restaurants.csv", sep = ";", header = FALSE, quote = "", stringsAsFactors=FALSE,
                        col.names = c("restid", "address1", "address2", "address3",
                                      "city", "county", "state", "zip", "lon", "lat",
                                      "open", "tempclose", "reopen", "close"))
head(restaurants)
length(unique(restaurants$restid))

### house cleaning

In [ ]:
sapply(restaurants, class) # check variables types
restaurants[, c(1, 9:10)] <- lapply(restaurants[, c(1, 9:10)], as.numeric) # convert geo coordinates to numeric
#restaurants[(100:120), ] #sanity check

convert_to_date <- function(x) {
    # replace 0000-00-00 dates as NA
    x[x=="0000-00-00"] <- NA
    # convert characters to dates
    x <- as.Date(x)
    return(x)
    print(class(x))
}
restaurants[, 11:14] <- lapply(restaurants[, 11:14], convert_to_date)
rm(convert_to_date)
sapply(restaurants, class) #check classes again
restaurants$address1[restaurants$address1=="N/A"] <- NA
restaurants$city[restaurants$city=="N/A"] <- NA
restaurants$zip[restaurants$zip==""] <- NA
restaurants <- restaurants[order(restaurants$open, restaurants$close), ]

In [65]:
restaurants[900:920, c(1:2, 5, 7:14)] #sanity check

In [ ]:
# create index for restaurant status
# closed, planned, open
restaurants$status <- NULL
length(restaurants$restid)
restaurants$status[is.na(restaurants$close) & !is.na(restaurants$open)] <- "open"
restaurants$status[!is.na(restaurants$close)] <- "closed"
restaurants$status[is.na(restaurants$close) & is.na(restaurants$open)] <- "planned"
table(restaurants$status)

In [ ]:
# clean up address data
colnames(restaurants)[c(2:4, 8)] <- c("street1", "street2", "street3", "longzip")
restaurants$longzip[restaurants$longzip=="NA" ] <- NA
class(restaurants$longzip)
restaurants$zip <- substr(restaurants$longzip, 1, 5)
restaurants$zip <- as.numeric(restaurants$zip)
restaurants$address <- paste(restaurants$street1, restaurants$city, paste(restaurants$state, restaurants$zip),
                             sep=", ")
restaurants$address <- sub(pattern=" NA$", replacement="", restaurants$address)

In [171]:
restaurants$longzip <- trimws(restaurants$longzip, "both")

In [172]:
restaurants <- restaurants[, c("restid", "address", "street1", "street2", "street3", "city", "county",
                              "state", "zip", "lon", "lat", "status", "open", "tempclose", "reopen", "close",
                               "longzip")]

In [250]:
names(restaurants)
head(restaurants)

In [251]:
table(restaurants$state)

In [174]:
restaurants$state <- trimws(restaurants$state, "both")

In [175]:
restaurants$state[restaurants$state=="DIST OF COLUMBIA"] <- "DC"
restaurants$state[restaurants$state=="LOS ANGELES"] <- "CA"
restaurants$state[restaurants$state=="ALLEGANY"] <- "NY"
restaurants$state[restaurants$state=="'"] <- ""
restaurants$state[restaurants$state=="JEFFERSON"] <- "KY"

In [252]:
table(restaurants$state[restaurants$status=="open"])

In [178]:
restaurants$tempclose_time <- restaurants$reopen - restaurants$tempclose

In [253]:
head(restaurants[!is.na(restaurants$tempclose_time), ])

In [ ]:
length(restaurants$restid[!is.na(restaurants$tempclose_time)])
summary(as.numeric(restaurants$tempclose_time))

### clean up longitude and latitude data

In [ ]:
summary(restaurants$lon)

In [ ]:
summary(restaurants$lon[restaurants$lon>=0])

### import lon/lat geocoded by census bureau

In [182]:
geocode_all <- c(0)
for (i in 1:2) {
    geocode <- read.csv(paste("geocoding-restaurants/GeocodeResults", i, ".csv", sep=""),
                        header=FALSE, stringsAsFactors=FALSE)
    geocode <- geocode[order(geocode$V1), ]
    
    geocode_all <- rbind(geocode_all, geocode)
}
geocode_all <- geocode_all[-1, c(2:3, 6, 9:12)]
colnames(geocode_all)[1:7] <- c("address2", "match", "coords", "state", "county", "tract", "block")

In [ ]:
dim(restaurants)
dim(geocode_all)

In [184]:
restaurants <- cbind(restaurants, geocode_all)

In [ ]:
names(restaurants)
dim(restaurants)

In [189]:
#sapply(restaurants, class)
restaurants <- restaurants[, -c(19:20)]

In [195]:
restaurants$lon2 <- sapply(strsplit(restaurants$coords, ","), "[", 1)

In [197]:
restaurants$lat2 <- sapply(strsplit(restaurants$coords, ","), "[", 2)

In [207]:
restaurants$lon2 <- as.numeric(restaurants$lon2)
restaurants$lat2 <- as.numeric(restaurants$lat2)

In [228]:
head(restaurants[(restaurants$lat>0.0000 & is.na(restaurants$lat2)), c("lon", "lat", "lon2", "lat2")])
class(restaurants$lon)

[1] "numeric"

In [254]:
length(restaurants$restid[restaurants$lon<0.0000 & is.na(restaurants$lon2)])

In [221]:
restaurants$lon2 <- ifelse(test=(restaurants$lon<0.0000 & is.na(restaurants$lon2)),
                           restaurants$lon, restaurants$lon2)
restaurants$lat2 <- ifelse(test=(restaurants$lat>0.0000 & is.na(restaurants$lat2)),
                           restaurants$lat, restaurants$lat2)

In [231]:
restaurants$coords <- NULL

In [240]:
colnames(restaurants)[1:24] <- c("restid", "address", "street1", "street2", "street3", "city", "county", 
                                 "state", "zip", "lon_fromtb", "lat_fromtb", "status", "open", "tempclose",
                                 "reopen", "close",
                                 "longzip", "tempclose_time", "state_num", "county_num", "tract", "block",
                                 "lon", "lat")

In [244]:
restaurants <- restaurants[c("restid", "address", "city", "state", "zip",
                             "status", "open", "tempclose", "reopen", "close", "tempclose_time",
                             "lon", "lat",
                             "state_num", "county_num", "tract", "block",
                             "street1", "street2", "street3","county", "lon_fromtb", "lat_fromtb", "longzip")]

In [255]:
names(restaurants)

In [246]:
write.csv(x=restaurants, file="restaurants-clean.csv", row.names=FALSE)